In [ ]:
# Guarded Setup
DRY_RUN = True
from notebooks._utils.common import *
CLI_OK = shell_available('forensic-cli')
LAB_ID = '60_memory_and_registry'
LAB_ROOT = lab_root(LAB_ID)
print(f'CLI available: {CLI_OK}')
print(f'Artifacts root: {LAB_ROOT}')


# Lab 60 · Memory and Registry


This notebook is part of the guarded, dry-run-first forensic lab suite.


> Placeholder skeleton: detailed lab content will be added in a later iteration.
